In [3]:
import sys
sys.path.append("/home/joao/Documents/gits/alquimodelia/")

In [4]:
from alquimodelia import alquimodelia as alqm
import importlib 
importlib.reload(alqm)

<module 'alquimodelia.alquimodelia' from '/home/joao/Documents/gits/alquimodelia/alquimodelia/alquimodelia.py'>

In [5]:
model = alqm.ModelMagia('3D_UNET')

sss
bef (None, 1, 140, 140, 1)
want (140, 140, 1)
(None, 140, 140, 1)
{"class_name": "Functional", "config": {"name": "model", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 8, 140, 140, 10], "dtype": "float32", "sparse": false, "ragged": false, "name": "input"}, "name": "input", "inbound_nodes": []}, {"class_name": "Conv3D", "config": {"name": "conv3d", "trainable": true, "dtype": "float32", "filters": 16, "kernel_size": [3, 3, 3], "strides": [1, 1, 1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1, 1, 1], "groups": 1, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "HeNormal", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "conv3d", "inbound_nodes": [[["input", 0, 0, {}]]]}, {"class_name": "BatchNormalization", "config

In [4]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [20]:
model = alqm.ModelMagia('2D_UNET')

sss
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 140, 140, 10 0                                            
__________________________________________________________________________________________________
conv2d_66 (Conv2D)              (None, 140, 140, 16) 1456        input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_60 (BatchNo (None, 140, 140, 16) 64          conv2d_66[0][0]                  
__________________________________________________________________________________________________
activation_60 (Activation)      (None, 140, 140, 16) 0           batch_normalization_60[0][0]     
________________________________________________________________________________________

In [36]:
import tensorflow as tf
X = tf.random.uniform(shape=(1, 224, 224, 1))

In [47]:
model = alqm.ModelMagia('ResNet')

resnet


TypeError: res_net() takes 0 positional arguments but 1 was given

In [3]:
import tensorflow as tf
from d2l import tensorflow as d2l

class Residual(tf.keras.Model):  #@save
    """The Residual block of ResNet."""
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(num_channels, padding='same',
                                            kernel_size=3, strides=strides)
        self.conv2 = tf.keras.layers.Conv2D(num_channels, kernel_size=3,
                                            padding='same')
        self.conv3 = None
        if use_1x1conv:
            self.conv3 = tf.keras.layers.Conv2D(num_channels, kernel_size=1,
                                                strides=strides)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()

    def call(self, X):
        Y = tf.keras.activations.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3 is not None:
            X = self.conv3(X)
        Y += X
        return tf.keras.activations.relu(Y)



class ResnetBlock(tf.keras.layers.Layer):
    def __init__(self, num_channels, num_residuals, first_block=False,
                 **kwargs):
        super(ResnetBlock, self).__init__(**kwargs)
        self.residual_layers = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                self.residual_layers.append(
                    Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                self.residual_layers.append(Residual(num_channels))

    def call(self, X):
        for layer in self.residual_layers.layers:
            X = layer(X)
        return X

In [27]:
def net():
    return tf.keras.Sequential([
        # The following layers are the same as b1 that we created earlier
        tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
        # The following layers are the same as b2, b3, b4, and b5 that we
        # created earlier
        ResnetBlock(64, 2, first_block=True),
        ResnetBlock(128, 2),
        ResnetBlock(256, 2),
        ResnetBlock(512, 2),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(units=1)])

In [ ]:
X = tf.random.uniform(shape=(1, 128, 128, 10))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

In [28]:
model = net()

In [29]:
# N, img, channel
model.build(input_shape=(None,128,128,10))

In [30]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_100 (Conv2D)          (None, 64, 64, 64)        31424     
_________________________________________________________________
batch_normalization_85 (Batc (None, 64, 64, 64)        256       
_________________________________________________________________
activation_5 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
resnet_block_20 (ResnetBlock (None, 32, 32, 64)        148736    
_________________________________________________________________
resnet_block_21 (ResnetBlock (None, 16, 16, 128)       526976    
_________________________________________________________________
resnet_block_22 (ResnetBlock (None, 8, 8, 256)        